# 🍓 Strawberry Vision – Google Colab Tam Kılavuz

Bu notebook, Strawberry Vision projesini Google Colab'de çalıştırmak için kapsamlı bir kılavuzdur.

## 📋 İçerik
1. **Kurulum**: Bağımlılıklar ve proje yapısı
2. **Dataset Hazırlama**: Roboflow'dan indirme ve etiketleme
3. **Model Eğitimi**: YOLOv8 eğitimi
4. **Inference**: Eğitilmiş model ile tahmin
5. **Değerlendirme**: Model performans analizi
6. **Testler**: Birim ve entegrasyon testleri

## 🚀 Hızlı Başlangıç
- **Sadece Inference**: Bölüm 1 ve 4'ü çalıştırın
- **Tam Eğitim Pipeline**: Tüm bölümleri sırayla çalıştırın

In [ ]:
# 1. KURULUM - Bağımlılıkları Yükle
print("📦 Bağımlılıklar yükleniyor...")

!pip install -q ultralytics>=8.0.0
!pip install -q opencv-python>=4.8.0
!pip install -q numpy>=1.24.0
!pip install -q matplotlib>=3.7.0
!pip install -q roboflow>=1.1.0
!pip install -q albumentations>=1.3.0
!pip install -q pyyaml>=6.0
!pip install -q pytest>=7.4.0
!pip install -q pytest-cov>=4.1.0

print("✅ Bağımlılıklar yüklendi!")

In [ ]:
# 2. PROJE YAPISINI KONTROL ET
import os
from pathlib import Path

project_root = Path.cwd()
print(f"📁 Proje dizini: {project_root}")

required_dirs = ["strawberry_vision", "configs", "scripts", "tests", "docs", "datasets"]
for dir_name in required_dirs:
    dir_path = project_root / dir_name
    dir_path.mkdir(exist_ok=True)
    print(f"✅ {dir_name}/")

print("✅ Proje yapısı hazır!")

## 📦 Bölüm 2: Dataset Hazırlama

In [ ]:
# 2.1. Roboflow'dan Dataset İndir
ROBOFLOW_API_KEY = "YOUR_API_KEY_HERE"
WORKSPACE = "strawberry"
PROJECT = "ripeness"
VERSION = 1

if ROBOFLOW_API_KEY != "YOUR_API_KEY_HERE":
    !python scripts/download_dataset.py --api-key {ROBOFLOW_API_KEY} --workspace {WORKSPACE} --project {PROJECT} --version {VERSION} --output datasets/roboflow
else:
    print("⚠️  API key girilmedi")

In [ ]:
# 2.2. Sınıf Etiketlerini Güncelle
!python scripts/relabel_dataset.py --input datasets/roboflow --output datasets/processed
print("✅ Standart sınıflar: strawberry_ripe, strawberry_semi_ripe, strawberry_unripe")

In [ ]:
# 2.3. Augmentation (Opsiyonel)
APPLY_AUGMENTATION = False
AUGMENTATION_FACTOR = 2

if APPLY_AUGMENTATION:
    !python scripts/augment_dataset.py --input datasets/processed --output datasets/augmented --factor {AUGMENTATION_FACTOR}
    DATASET_PATH = "datasets/augmented"
else:
    DATASET_PATH = "datasets/processed"

print(f"Dataset: {DATASET_PATH}")

## 🎓 Bölüm 3: Model Eğitimi

In [ ]:
# 3.1. Eğitim Parametreleri
MODEL = "yolov8n.pt"
EPOCHS = 50
BATCH_SIZE = 16
IMAGE_SIZE = 640
EXPERIMENT_NAME = "strawberry_colab_exp"

print(f"Model: {MODEL}, Epochs: {EPOCHS}, Batch: {BATCH_SIZE}")

In [ ]:
# 3.2. Model Eğitimi
!python scripts/train_yolo.py --data {DATASET_PATH}/data.yaml --model {MODEL} --epochs {EPOCHS} --batch {BATCH_SIZE} --imgsz {IMAGE_SIZE} --name {EXPERIMENT_NAME} --device 0
print(f"✅ Model: runs/train/{EXPERIMENT_NAME}/weights/best.pt")

In [ ]:
# 3.3. Eğitim Sonuçları
from IPython.display import Image, display
results_path = f"runs/train/{EXPERIMENT_NAME}"
try:
    display(Image(filename=f"{results_path}/results.png"))
    display(Image(filename=f"{results_path}/confusion_matrix.png"))
except:
    print("⚠️  Grafikler bulunamadı")

## 🔍 Bölüm 4: Inference

In [ ]:
# 4.1. Model Yolu
TRAINED_MODEL = f"runs/train/{EXPERIMENT_NAME}/weights/best.pt"
print(f"Model: {TRAINED_MODEL}")

In [ ]:
# 4.2. Tek Görüntü Inference
from strawberry_vision.application.pipeline import InferencePipeline
from strawberry_vision.infrastructure.detectors import YOLODetector
from strawberry_vision.infrastructure.sources import ImageSource
import cv2
import matplotlib.pyplot as plt

TEST_IMAGE = "sample.jpg"

if os.path.exists(TEST_IMAGE) and os.path.exists(TRAINED_MODEL):
    detector = YOLODetector(model_path=TRAINED_MODEL)
    pipeline = InferencePipeline(detector=detector, enable_logging=False)
    source = ImageSource(image_path=TEST_IMAGE)
    
    for frame in source:
        result = pipeline.run(frame)
        print(f"Toplam: {result.total}, Dağılım: {result.counts}")
        
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f"Tespit: {result.total} çilek")
        plt.show()
else:
    print("⚠️  Görüntü veya model bulunamadı")

## 📊 Bölüm 5: Değerlendirme

In [ ]:
# 5.1. Model Değerlendirme
if os.path.exists(TRAINED_MODEL):
    !python scripts/evaluate_model.py --model {TRAINED_MODEL} --data {DATASET_PATH}/data.yaml --split val
else:
    print("⚠️  Model bulunamadı")

## 🧪 Bölüm 6: Testler

In [ ]:
# 6.1. Testleri Çalıştır
!pytest tests/ -v
!pytest --cov=strawberry_vision tests/ --cov-report=term-missing

## 💾 Bölüm 7: Sonuçları Kaydet

In [ ]:
# 7.1. Zip ve İndir
import shutil
from google.colab import files

if os.path.exists(f"runs/train/{EXPERIMENT_NAME}"):
    shutil.make_archive(f"strawberry_results_{EXPERIMENT_NAME}", 'zip', f"runs/train/{EXPERIMENT_NAME}")
    files.download(f"strawberry_results_{EXPERIMENT_NAME}.zip")
    print("✅ İndirme tamamlandı")
else:
    print("⚠️  Sonuç klasörü bulunamadı")

## 📝 Notlar

### GPU Aktivasyonu
Runtime > Change runtime type > GPU

### Önerilen Parametreler
- **Hızlı Test**: yolov8n.pt, 20 epoch, batch 16
- **Orta Kalite**: yolov8s.pt, 50 epoch, batch 16
- **Yüksek Kalite**: yolov8m.pt, 100 epoch, batch 8-16

### Sorun Giderme
- **Out of Memory**: Batch size düşürün
- **Yavaş Eğitim**: GPU kontrolü yapın

### Linkler
- [Dokümantasyon](docs/INDEX.md)
- [Roboflow Dataset](docs/3-RoboflowDatasetKullanimi.md)